In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from firebase import firebase
import time
import pandas as pd
import numpy as np

In [2]:
firebase = firebase.FirebaseApplication('https://hotelwishlist-86f4f.firebaseio.com/', None)

In [4]:
browser = webdriver.Chrome("C:\\Users\\saurabh\\Desktop\\MMT-Flight\\chromedriver.exe")

In [5]:
df = pd.read_csv(r'data/city_codes.csv')

In [6]:
result = firebase.get('/info/','')
values_view = result.values()
value_iterator = iter(values_view)
first_value = next(value_iterator)
print(first_value)

{'inDate': '08/07/2020', 'location': 'Delhi', 'outDate': '09/07/2020'}


In [7]:
#location = 'Delhi'
location = first_value['location']
location = location.lower().capitalize()
locationCode = df['iata_code'][df[df['city'] == location].index.tolist()].tolist()[0]

In [8]:
#checkin = '08/07/2020' #dd/mm/yyyy
checkin = first_value['inDate']
tempDate = checkin.split('/')
checkin_url = tempDate[1] + tempDate[0] + tempDate[2]

In [9]:
#checkout = '09/07/2020' #dd/mm/yyyy
checkout = first_value['outDate']
tempDate = checkout.split('/')
checkout_url = tempDate[1] + tempDate[0] + tempDate[2]

In [10]:
browser.implicitly_wait(20)

In [11]:
url = 'https://www.makemytrip.com/hotels/hotel-listing/?checkin=' + checkin_url + '&checkout=' + checkout_url + '&locusId=CT' + locationCode + '&locusType=city&city=CT' + locationCode + '&country=IN&searchText=' + location + '&roomStayQualifier=2e0e&_uCurrency=INR&reference=hotel&type=city'

In [12]:
browser.get(url)

In [13]:
# Scroll the page till bottom to get full data available in the DOM.
print ("Scrolling document upto bottom ...")
for j in range(1, 10):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Find the document body and get its inner HTML for processing in BeautifulSoup parser.
body = browser.find_element_by_tag_name("body").get_attribute("innerHTML")

Scrolling document upto bottom ...


In [14]:
print("Getting data from DOM ...")
soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup

Getting data from DOM ...


In [15]:
pHotelName = soupBody.findAll("p", {"class": "latoBlack font22 blackText appendBottom12"}) 

In [16]:
hotelsNameData=[]
for j in range(0, len(pHotelName)):
    hotelsNameData.append(pHotelName[j].text)

In [17]:
pHotelPrice = soupBody.findAll("p", {"class": "latoBlack font26 blackText appendBottom5"}) 

In [18]:
hotelsPriceData=[]
for j in range(0, len(pHotelPrice)):
    hotelsPriceData.append(pHotelPrice[j].text)

In [19]:
scraped_data = []

for i in range(len(hotelsPriceData)):

    # initialize the dictionary
    card_details = {}

    # add data to the dictionary
    card_details['hotel_name'] = hotelsNameData[i]
    card_details['room_price'] = hotelsPriceData[i]

    # append the scraped data to the list
    scraped_data.append(card_details)

# create a data frame from the list of dictionaries
dataFrame = pd.DataFrame.from_dict(scraped_data)

In [20]:
dataFrame.insert(0, 'location', location)
dataFrame.insert(0, 'index', range(0, len(dataFrame)))

In [21]:
dataFrame

,index,location,hotel_name,room_price
0,0,Delhi,Red Fox Hotel Delhi Airport,"INR2,631"
1,1,Delhi,The Piccadily,"INR4,653"
2,2,Delhi,Hotel Aero Star,"INR2,251"
3,3,Delhi,The Park New Delhi,"INR5,499"
4,4,Delhi,Hotel Arch,"INR1,667"


In [22]:
json_records = dataFrame.to_json(r'result.json',orient ='records') 

In [23]:
json_records = dataFrame.to_dict(orient ='records')
print(json_records)

[{'index': 0, 'location': 'Delhi', 'hotel_name': 'Red Fox Hotel Delhi Airport', 'room_price': 'INR2,631'}, {'index': 1, 'location': 'Delhi', 'hotel_name': 'The Piccadily', 'room_price': 'INR4,653'}, {'index': 2, 'location': 'Delhi', 'hotel_name': 'Hotel Aero Star', 'room_price': 'INR2,251'}, {'index': 3, 'location': 'Delhi', 'hotel_name': 'The Park New Delhi', 'room_price': 'INR5,499'}, {'index': 4, 'location': 'Delhi', 'hotel_name': 'Hotel Arch', 'room_price': 'INR1,667'}]


In [24]:
firebase.delete('/hotel/', '')
for record in json_records:
    print(record)
    result = firebase.post('/hotel/',record)

{'index': 0, 'location': 'Delhi', 'hotel_name': 'Red Fox Hotel Delhi Airport', 'room_price': 'INR2,631'}
{'index': 1, 'location': 'Delhi', 'hotel_name': 'The Piccadily', 'room_price': 'INR4,653'}
{'index': 2, 'location': 'Delhi', 'hotel_name': 'Hotel Aero Star', 'room_price': 'INR2,251'}
{'index': 3, 'location': 'Delhi', 'hotel_name': 'The Park New Delhi', 'room_price': 'INR5,499'}
{'index': 4, 'location': 'Delhi', 'hotel_name': 'Hotel Arch', 'room_price': 'INR1,667'}


In [25]:
result = firebase.get('/hotel/','')
print(result)

{'-MBIGLHQUmuEP_g_5txQ': {'hotel_name': 'Red Fox Hotel Delhi Airport', 'index': 0, 'location': 'Delhi', 'room_price': 'INR2,631'}, '-MBIGLc9DA-NBTg-TmOv': {'hotel_name': 'The Piccadily', 'index': 1, 'location': 'Delhi', 'room_price': 'INR4,653'}, '-MBIGLx--8Snavt7oU14': {'hotel_name': 'Hotel Aero Star', 'index': 2, 'location': 'Delhi', 'room_price': 'INR2,251'}, '-MBIGMIWbVv0LBU55PkA': {'hotel_name': 'The Park New Delhi', 'index': 3, 'location': 'Delhi', 'room_price': 'INR5,499'}, '-MBIGMeSDv0cYQmOdp4_': {'hotel_name': 'Hotel Arch', 'index': 4, 'location': 'Delhi', 'room_price': 'INR1,667'}}
